In [1]:
!ls

best_openvino_model			   deepsparse_prune_quant.ipynb
best_opset13.onnx			   detect4
best_opset19.onnx			   lab3_py3.11.ipynb
best_prune_75_quantized_int4_opset13.onnx  onnx_quant_prune.ipynb
best_prune_75_quantized_int8_opset13.onnx  onnx_quant_prune-opset19.ipynb
best_prune_opset13.onnx			   prune_.ipynb
best_prune.pt				   runs
best.pt


In [2]:
from ultralytics import YOLO

model = YOLO("best_prune.pt") 
model.export(format='onnx')

Ultralytics 8.3.39 🚀 Python-3.12.7 torch-2.4.1+cpu CPU (AMD Ryzen 7 7730U with Radeon Graphics)
Model summary (fused): 268 layers, 68,156,310 parameters, 0 gradients, 257.6 GFLOPs

PyTorch: starting from 'best_prune.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 38, 8400) (130.4 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.43...
ONNX: export success ✅ 4.3s, saved as 'best_prune.onnx' (260.2 MB)

Export complete (6.4s)
Results saved to /home/void/ml_oleg/lab3
Predict:         yolo predict task=detect model=best_prune.onnx imgsz=640  
Validate:        yolo val task=detect model=best_prune.onnx imgsz=640 data=test.yaml  
Visualize:       https://netron.app


'best_prune.onnx'

In [3]:
!mv best_prune.onnx best_prune_opset19.onnx

In [4]:
import numpy as np
from onnxruntime.quantization import CalibrationDataReader, quantize_static, QuantType, QuantFormat
import os 
import cv2
class ImageCalibrationDataReader(CalibrationDataReader):
    def __init__(self, image_paths):
        self.image_paths = image_paths
        self.idx = 0
        self.input_name = "images"

    def preprocess(self, frame):
        frame = cv2.imread(frame)
        X = cv2.resize(frame, (640, 640))
        image_data = np.array(X).astype(np.float32) / 255.0  
        image_data = np.transpose(image_data, (2, 0, 1))  
        image_data = np.expand_dims(image_data, axis=0)  
        return image_data

    def get_next(self):
        if self.idx >= len(self.image_paths):
            return None

        image_path = self.image_paths[self.idx]
        input_data = self.preprocess(image_path)
        self.idx += 1
        return {self.input_name: input_data}

calibration_image_paths  = os.listdir('../annotations/images/test/')[:100]
calibration_image_paths =list(map(lambda x: f'../annotations/images/test/{x}',calibration_image_paths))


calibration_data_reader = ImageCalibrationDataReader(calibration_image_paths)

In [5]:
quantize_static('best_prune_opset19.onnx', "best_prune_75_quantized_int8_opset19.onnx",
                weight_type=QuantType.QInt8,
                activation_type=QuantType.QUInt8,
                calibration_data_reader=calibration_data_reader,
                quant_format=QuantFormat.QDQ,
                nodes_to_exclude=['/model.22/Concat_3', '/model.22/Split', '/model.22/Sigmoid'
                                 '/model.22/dfl/Reshape', '/model.22/dfl/Transpose', '/model.22/dfl/Softmax', 
                                 '/model.22/dfl/conv/Conv', '/model.22/dfl/Reshape_1', '/model.22/Slice_1',
                                 '/model.22/Slice', '/model.22/Add_1', '/model.22/Sub', '/model.22/Div_1',
                                  '/model.22/Concat_4', '/model.22/Mul_2', '/model.22/Concat_5'],
                per_channel=False,
                reduce_range=True,)

In [6]:
calibration_data_reader = ImageCalibrationDataReader(calibration_image_paths)
quantize_static('best_prune_opset19.onnx', "best_prune_75_quantized_int4_opset19.onnx",
                weight_type=QuantType.QInt4,
                activation_type=QuantType.QUInt4,
                calibration_data_reader=calibration_data_reader,
                quant_format=QuantFormat.QDQ,
                nodes_to_exclude=['/model.22/Concat_3', '/model.22/Split', '/model.22/Sigmoid'
                                 '/model.22/dfl/Reshape', '/model.22/dfl/Transpose', '/model.22/dfl/Softmax', 
                                 '/model.22/dfl/conv/Conv', '/model.22/dfl/Reshape_1', '/model.22/Slice_1',
                                 '/model.22/Slice', '/model.22/Add_1', '/model.22/Sub', '/model.22/Div_1',
                                  '/model.22/Concat_4', '/model.22/Mul_2', '/model.22/Concat_5'],
                per_channel=False,
                reduce_range=True,)